In [2]:
import pandas as pd
import mysql.connector
import uuid
import re
import sys

In [3]:
MYSQL_HOST = "localhost"
MYSQL_USER = "root"  
MYSQL_PASSWORD = ""
MYSQL_DATABASE = "video_games_db"

In [4]:
df_vgsales = pd.read_csv("vgsales.csv")
games = pd.read_csv("games.csv")


In [5]:
games

,Unnamed: 0,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
0,0,Elden Ring,25-Feb-22,"['Bandai Namco Entertainment', 'FromSoftware']",4.5,3.9K,3.9K,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...",17K,3.8K,4.6K,4.8K
1,1,Hades,10-Dec-19,['Supergiant Games'],4.3,2.9K,2.9K,"['Adventure', 'Brawler', 'Indie', 'RPG']",A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...,21K,3.2K,6.3K,3.6K
2,2,The Legend of Zelda: Breath of the Wild,03-Mar-17,"['Nintendo', 'Nintendo EPD Production Group No...",4.4,4.3K,4.3K,"['Adventure', 'RPG']",The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...,30K,2.5K,5K,2.6K
3,3,Undertale,15-Sep-15,"['tobyfox', '8-4']",4.2,3.5K,3.5K,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...","A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...,28K,679,4.9K,1.8K
4,4,Hollow Knight,24-Feb-17,['Team Cherry'],4.4,3K,3K,"['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with...",21K,2.4K,8.3K,2.3K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,1507,Back to the Future: The Game,22-Dec-10,['Telltale Games'],3.2,94,94,"['Adventure', 'Point-and-Click']",Back to the Future: The Game is one of Telltal...,['Very enjoyable game. The story adds onto the...,763,5,223,67
1508,1508,Team Sonic Racing,21-May-19,"['Sumo Digital', 'Sega']",2.9,264,264,"['Arcade', 'Racing']",Team Sonic Racing combines the best elements o...,"['jogo morto mas bom', 'not my cup of tea', ""C...",1.5K,49,413,107
1509,1509,Dragon's Dogma,22-May-12,['Capcom'],3.7,210,210,"['Brawler', 'RPG']","Set in a huge open world, Dragon’s Dogma: Dark...","['Underrated.', 'A grandes rasgos, es como un ...",1.1K,45,487,206
1510,1510,Baldur's Gate 3,06-Oct-20,['Larian Studios'],4.1,165,165,"['Adventure', 'RPG', 'Strategy', 'Tactical', '...","An ancient evil has returned to Baldur's Gate,...",['Bu türe bu oyunla girmeye çalışmak hataydı s...,269,79,388,602


In [6]:
df_vgsales

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


In [7]:
# # --- Preprocessing and Cleaning for vgsales.csv ---
# sales.dropna(subset=['Year', 'Publisher', 'Name', 'Platform', 'Genre'], inplace=True)
# sales.drop_duplicates(inplace=True)
# sales['Year'] = pd.to_numeric(sales['Year'], errors='coerce').astype('Int64')
# sales.dropna(subset=['Year'], inplace=True)
# sales_cols = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
# for col in sales_cols:
#     sales[col] = pd.to_numeric(sales[col], errors='coerce')
# sales.dropna(subset=sales_cols, inplace=True)
# sales['Platform'] = sales['Platform'].str.lower().str.strip()
# sales['Genre'] = sales['Genre'].str.lower().str.strip()
# sales['Publisher'] = sales['Publisher'].str.lower().str.strip()

df_vgsales.dropna()
games.dropna()


,Unnamed: 0,Title,Release Date,Team,Rating,Times Listed,Number of Reviews,Genres,Summary,Reviews,Plays,Playing,Backlogs,Wishlist
0,0,Elden Ring,25-Feb-22,"['Bandai Namco Entertainment', 'FromSoftware']",4.5,3.9K,3.9K,"['Adventure', 'RPG']","Elden Ring is a fantasy, action and open world...","[""The first playthrough of elden ring is one o...",17K,3.8K,4.6K,4.8K
1,1,Hades,10-Dec-19,['Supergiant Games'],4.3,2.9K,2.9K,"['Adventure', 'Brawler', 'Indie', 'RPG']",A rogue-lite hack and slash dungeon crawler in...,['convinced this is a roguelike for people who...,21K,3.2K,6.3K,3.6K
2,2,The Legend of Zelda: Breath of the Wild,03-Mar-17,"['Nintendo', 'Nintendo EPD Production Group No...",4.4,4.3K,4.3K,"['Adventure', 'RPG']",The Legend of Zelda: Breath of the Wild is the...,['This game is the game (that is not CS:GO) th...,30K,2.5K,5K,2.6K
3,3,Undertale,15-Sep-15,"['tobyfox', '8-4']",4.2,3.5K,3.5K,"['Adventure', 'Indie', 'RPG', 'Turn Based Stra...","A small child falls into the Underground, wher...",['soundtrack is tied for #1 with nier automata...,28K,679,4.9K,1.8K
4,4,Hollow Knight,24-Feb-17,['Team Cherry'],4.4,3K,3K,"['Adventure', 'Indie', 'Platform']",A 2D metroidvania with an emphasis on close co...,"[""this games worldbuilding is incredible, with...",21K,2.4K,8.3K,2.3K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1507,1507,Back to the Future: The Game,22-Dec-10,['Telltale Games'],3.2,94,94,"['Adventure', 'Point-and-Click']",Back to the Future: The Game is one of Telltal...,['Very enjoyable game. The story adds onto the...,763,5,223,67
1508,1508,Team Sonic Racing,21-May-19,"['Sumo Digital', 'Sega']",2.9,264,264,"['Arcade', 'Racing']",Team Sonic Racing combines the best elements o...,"['jogo morto mas bom', 'not my cup of tea', ""C...",1.5K,49,413,107
1509,1509,Dragon's Dogma,22-May-12,['Capcom'],3.7,210,210,"['Brawler', 'RPG']","Set in a huge open world, Dragon’s Dogma: Dark...","['Underrated.', 'A grandes rasgos, es como un ...",1.1K,45,487,206
1510,1510,Baldur's Gate 3,06-Oct-20,['Larian Studios'],4.1,165,165,"['Adventure', 'RPG', 'Strategy', 'Tactical', '...","An ancient evil has returned to Baldur's Gate,...",['Bu türe bu oyunla girmeye çalışmak hataydı s...,269,79,388,602


In [8]:
# # --- Preprocessing and Cleaning for games.csv ---
# games['Rating'] = games['Rating'].apply(
#     lambda x: float(re.sub(r'[^0-9.]', '', str(x))) if re.sub(r'[^0-9.]', '', str(x)) else None
# )
# games.dropna(subset=['Rating', 'Release Date', 'Number of Reviews'], inplace=True)
# # Avoid columns with lists when dropping duplicates
# games.drop_duplicates(subset=['Title', 'Release Date', 'Rating'], inplace=True)
# games['Release Date'] = pd.to_datetime(games['Release Date'], format="%b %d, %Y", errors='coerce')
# games['Number of Reviews'] = pd.to_numeric(
#     games['Number of Reviews'].apply(lambda x: re.sub(r'[^0-9]', '', str(x))),
#     errors='coerce'
# ).astype('Int64')
# games.dropna(subset=['Release Date', 'Number of Reviews'], inplace=True)
# games['Title'] = games['Title'].str.lower().str.strip()

In [9]:
# Create a unique game_id for joining
# df_vgsales['game_id'] = [str(uuid.uuid4()) for _ in range(len(sales))]
# games['game_id'] = [str(uuid.uuid4()) for _ in range(len(games))]

print("Data cleaning complete!")

Data cleaning complete!


In [10]:
conn = mysql.connector.connect(
            host=MYSQL_HOST,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD
        )
cursor = conn.cursor()

In [ ]:
cursor.execute(f"CREATE DATABASE IF NOT EXISTS video_games_db")
cursor.execute(f"USE video_games_db")

cursor.execute("""
CREATE TABLE IF NOT EXIST games (
    game_id VARCHAR(36) PRIMARY KEY,
    title VARCHAR(255) NOT NULL,
    release_date DATE
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXIST sales_data (
    rank INT PRIMARY KEY,
    game_id VARCHAR(36),
    platform VARCHAR(50),
    year INT,
    genre VARCHAR(50),
    publisher VARCHAR(255),
    na_sales FLOAT,
    eu_sales FLOAT,
    jp_sales FLOAT,
    other_sales FLOAT,
    global_sales FLOAT,
) 
""")
cursor.execute("""
CREATE TABLE ratings_data (
    game_id VARCHAR(36) PRIMARY KEY,
    rating FLOAT,
    number_of_reviews INT,
    genres TEXT,
    summary TEXT,
    FOREIGN KEY (game_id) REFERENCES games(game_id)
)""")
print("Tables created successfully.")

cursor.close()
conn.close()

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'EXIST games (
    game_id VARCHAR(36) PRIMARY KEY,
    title VARCHAR(255) NOT NU' at line 1